In [2]:
nome_arquivo_original=r'C:\Users\Marcos\Desktop\tic\bases\DANTES_Large.conllu'
nome_arquivo_tratado='DANTES_Large_treated.conllu'

In [3]:
def preprocess_conllu(file_path):
    # Read the file content
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove empty lines and lines that do not follow the format
    cleaned_lines = []
    for line in lines:
        # Remove leading/trailing whitespace and skip empty lines
        line = line.strip()
        if line == "" or line.startswith("#"):
            continue  # Skip empty lines and comments
        if "\t" not in line and "  " not in line:
            print(f"Skipping invalid line: {line}")
            continue  # Skip lines without tabs or two spaces

        # Replace multiple spaces with a single tab to ensure correct formatting
        line = "\t".join(line.split())
        cleaned_lines.append(line)

    # Join cleaned lines back into a single string
    cleaned_content = "\n".join(cleaned_lines)
    return cleaned_content


In [4]:
# Função para consolidar linhas 'text=' e todas as subsequentes até encontrar uma linha de token
def preprocess_conllu_text(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', encoding='utf-8') as outfile:
        current_text = None  # Armazena o conteúdo da linha 'text=' consolidada

        for line in infile:
            # Se a linha começa com '# text =', inicializa a consolidação
            if line.startswith('# text ='):
                current_text = line.strip()  # Inicia a linha de 'text='
            elif current_text and not '\t' in line:
                # Adiciona conteúdo subsequente até a primeira linha de token
                current_text += " " + line.strip()
            else:
                # Escreve a linha 'text=' consolidada e redefine a variável
                if current_text:
                    outfile.write(current_text + "\n")
                    current_text = None  # Reseta o temporário
                # Escreve linhas de tokens ou outras linhas normais
                outfile.write(line)

        # Escreve a última linha consolidada de 'text=' se houver
        if current_text:
            outfile.write(current_text + "\n")

preprocess_conllu_text(nome_arquivo_original, 'intermediario.conllu')


In [5]:
def preprocess_conllu_and_remove_invalid_ids(caminho_arquivo_entrada, caminho_arquivo_saida):
    with open(caminho_arquivo_entrada, 'r', encoding='utf-8') as f:
        linhas = f.readlines()

    with open(caminho_arquivo_saida, 'w', encoding='utf-8') as f_out:
        for i, linha in enumerate(linhas, start=1):
            # Verifica se a linha é um token e se o ID é apenas um ponto
            if not linha.startswith("#") and linha.strip():
                campos = linha.split("\t")
                if campos[0].strip() == "." or campos[0].strip() == ".." or campos[0].strip() == "...":
                    print(f"Excluindo linha {i}: {linha.strip()}")
                    continue  # Pula a linha se o ID é um ponto

            # Escreve a linha no arquivo de saída se não foi excluída
            f_out.write(linha)

preprocess_conllu_and_remove_invalid_ids('intermediario.conllu', nome_arquivo_tratado)
